# Mean/Median Trajectory Data Filtering.

### Tutorial by : Mahmoud AbdelRahman
arch.mahmoud.ouf111[at]gmail.com


### First we import the required modules, especially random. 

In [2]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import os
import subprocess
import pandas as pd
import numpy as np
import random

%matplotlib inline

### Second, lets start generating a gps-like trajectory data. 
For a typical GPS trajectory data, error could be modelled as a two dimensional Gaussian white noise probability density with zero mean and diagonal covariance matrix R such that:
$$R = \begin{vmatrix} 
\sigma^2 & 0 \\
    0 & \sigma^2 
\end{vmatrix} 
$$
    
This is similar to adding zero mean and $\sigma$ standard deviation noise for each $x$, and $y$ direction separately to reflect the modelling inaccuracies.

In [ ]:
files = os.listdir("./imgs")

for i in files:
    if i[-3:] == "jpg":
        os.remove("./imgs/"+i)
        #print("./imgs/"+i + " Removed ... ")



pos = [[0.,100.]]
xs0 = [1.0]
ys0 = [50.0]

xs1 = [1.0]
ys1 = [50.]

gtx = [1.0]
gty = [50.]

gtx1 = []
gty1 = []


x0 = 0.
y0 = 50.

x1 = 0.
y1 = 50.


signx = 1.0
signy = 1.0


slidingWindowx = []
slidingWindowy = []

width = 200
height = 100

ok = True
n = 0
speedIndex = 0
speeds = np.random.uniform(0.2, 2.0, 50).tolist()
speed = 0.5
speedsign = 1
while ok:
    n+=1
    if x1 > width:
        break
        signx = -1.0

    if y1 > height:
        signy = -1.0

    if x1 < 0:
        signx = 1.0

    if y1 < 0 :
        signy = 1.0

    if float(n) %50.0 == 0:
        speedIndex+=1

    fig, ax = plt.subplots(2, 1, figsize=(10, 2.5))
    x1  =  xs1[-1]+ speeds[speedIndex] + np.random.normal(0., 2., 1).tolist()[0]
    y1 =  50+ np.random.normal(0., 2.0,1).tolist()[0]
    xs1.append(x1)
    ys1.append(y1)

    gtx1.append(xs1[-1]+ speeds[speedIndex])
    gty1.append(50.0)
    ax[0].cla()
    ax[0].set_xlim(0, width)
    ax[0].set_ylim(45, 55)

    ax[0].plot(xs1, ys1, color="black", alpha=0.8, linewidth=1.0, label="measured data")
    ax[0].plot(gtx1, gty1, color="red", linewidth=1.0, label="ground truth")
    p1 = mpatches.Circle((x1,y1), radius=1, color="red", label = "moving object")
    ax[0].add_patch(p1)
    ax[0].legend()

    
    if float(n)%20.0 == 0.:
        x0 = np.average(slidingWindowx)
        y0 = np.average(slidingWindowy)

        xs0.append(x0)
        ys0.append(y0)

        gtx.append(x0)
        gty.append(50.0)   

        slidingWindowx = []
        slidingWindowy = []
    else:
        slidingWindowx.append(x1)
        slidingWindowy.append(y1)

    ax[1].cla()

    ax[1].set_xlim(0, width)
    ax[1].set_ylim(45, 55)

    ax[1].plot(xs0, ys0, color="black", alpha=0.8, linewidth=1.0, label ="mean/median filter")
    ax[1].plot(slidingWindowx, slidingWindowy, color="blue", alpha=0.5, linewidth=1.0, label="sliding window")

    ax[1].plot(gtx, gty, color="red", linewidth=1.0, label="ground truth")
    p0 = mpatches.Circle((x0,y0), radius=1, color="red", label="moving object")
    ax[1].add_patch(p0)
    

    ax[1].legend()

    fig.savefig('./imgs/'+"{:03d}".format(n)+".jpg")
n = [-1]
plt.close()
files = os.listdir("./imgs")
for i in files:
    if i[-3:] == "gif":
        try:
            n.append(int(i.split(".")[0][-3:]))
        except:
            print( " no n ... ")


p = subprocess.Popen(["convert", "-delay", "10", "-loop", "0", "./imgs/*.jpg", "./imgs/image"+"{:03d}".format(max(n)+1)+".gif"])
p.wait()


print(files)
for i in files:
    if i[-3:] == "jpg":
        os.remove("./imgs/"+i)
        print("./imgs/"+i + " Removed ... ")


d = {"x":xs1, "y":ys1}

df = pd.DataFrame(d)
df.to_csv("./imgs/df2.csv", index=False)
print(df)

### The result : 
![](./imgs/image020.gif)